In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')

nltk.download('wordnet')

nltk.download('averaged_perceptron_tagger')

In [9]:
import pandas as pd
import numpy as np
import time
import scipy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem import PorterStemmer
from nltk import word_tokenize   
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [17]:
train = pd.read_csv('./train.csv',engine='python')
test = pd.read_csv('./test.csv',engine='python')
X_train = train.body
Y_train = train.subreddit
X_test = test.body

vectorizer = CountVectorizer()
normalizer_train = Normalizer()
tfidf = TfidfTransformer()


# transform categories into number
class_le = LabelEncoder()
Y_train = class_le.fit_transform(Y_train.values)

vectors_train = vectorizer.fit_transform(X_train)
vectors_test = vectorizer.transform(X_test)
vectors_train = vectors_train.todense()
vectors_test = vectors_test.todense()

onehot = OneHotEncoder(handle_unknown = 'ignore')
vectors_train = onehot.fit_transform(vectors_train)
vectors_test = onehot.transform(vectors_test)

#vectors_train = normalizer_train.transform(vectors_train)
#vectors_test = normalizer_train.transform(vectors_test)

print(vectorizer.get_feature_names())
print(vectors_train.shape)
print(vectors_test.shape)
#vectorizer_test = CountVectorizer(vocabulary=vectorizer_train.vocabulary_)
#vectorizer = CountVectorizer(vocabulary=vectorizer.vocabulary_)
#a=np.sum(vectors_train,axis=0)
#print(vectors_train.max())
#print(np.argmax(a))

['00', '000', '000inr', '000n', '001', '007', '00am', '00pm', '01', '01004', '01110', '012', '019', '01k009en', '02', '020', '02088', '021', '0261', '03', '03305e43', '0499', '04f6adee', '05', '062', '066', '07', '081', '09', '097in', '099in', '0c04786', '0c4a743960cc', '10', '100', '1000', '100000', '1002', '1003', '10051', '100567028', '100579313', '100579449', '1005g1', '100gb', '100k', '100v', '100x', '101', '1012', '10151', '1016', '1017', '101858977111', '10205989', '1021', '1027', '1029864920981110', '10300h', '1035g1', '1047', '1050', '1050ee', '1050ti', '10510u', '1056', '1060', '1065g7', '107', '1070', '10750h', '1080', '1080p', '10870', '10875h', '108mp', '1093', '10980hk', '10am', '10k', '10plus', '10pm', '10s', '10sfk', '10th', '10tk', '10ux', '10x', '10x3', '11', '110', '1100', '110k', '111', '1115g4', '112153', '1126', '113476', '1135g7', '114307', '114446', '115w', '1165g7', '116700', '116a', '117085', '1172', '1177', '118', '1183', '1186', '11am', '11ax', '11gen', '11h

In [11]:
# remove stop words
stop_words = text.ENGLISH_STOP_WORDS
vectorizer = CountVectorizer(stop_words = stop_words)
vectors_train_stop = vectorizer.fit_transform(X_train)
vectors_test_stop = vectorizer.transform(X_test)
vectors_train_stop= normalizer_train.transform(vectors_train_stop)
vectors_test_stop = normalizer_train.transform(vectors_test_stop)
print(vectors_train_stop.shape)

(1999, 15079)


In [ ]:
# tf-idf
tf_idf_vectorizer = TfidfVectorizer()
vectors_train_idf = tf_idf_vectorizer.fit_transform(X_train)
vectors_test_idf = tf_idf_vectorizer.transform(X_test)
vectors_train_idf= normalizer_train.transform(vectors_train_idf)
vectors_test_idf = normalizer_train.transform(vectors_test_idf)

In [ ]:
# stemming
class StemTokenizer:
     def __init__(self):
       self.wnl =PorterStemmer()
     def __call__(self, doc):
       return [self.wnl.stem(t) for t in word_tokenize(doc) if t.isalpha()]

vectorizer = CountVectorizer(tokenizer=StemTokenizer())
vectors_train_stem = vectorizer.fit_transform(X_train)
vectors_test_stem = vectorizer.transform(X_test)
vectors_train_stem= normalizer_train.transform(vectors_train_stem)
vectors_test_stem = normalizer_train.transform(vectors_test_stem)
print(vectors_train_stem.shape)

(1999, 8727)


In [ ]:
# Lemmatization
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
class New_LemmaTokenizer:
     def __init__(self):
       self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
       return [self.wnl.lemmatize(t,pos =get_wordnet_pos(t)) for t in word_tokenize(doc) if t.isalpha()]

vectorizer = CountVectorizer(tokenizer=New_LemmaTokenizer())
vectors_train_Lemma = vectorizer.fit_transform(X_train)
vectors_test_Lemma = vectorizer.transform(X_test)
vectors_train_Lemma= normalizer_train.transform(vectors_train_Lemma)
vectors_test_Lemma = normalizer_train.transform(vectors_test_Lemma)
print(vectors_train_Lemma.shape)

(1999, 10045)


In [ ]:
# stopwords tfidf punctuation lemmatization normalization
stop_words = text.ENGLISH_STOP_WORDS

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
class New_LemmaTokenizer:
     def __init__(self):
       self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
       return [self.wnl.lemmatize(t,pos =get_wordnet_pos(t)) for t in word_tokenize(doc) if t.isalpha()]

vectorizer = CountVectorizer(stop_words = stop_words,tokenizer=New_LemmaTokenizer())
vectors_train_stop_Lemma = vectorizer.fit_transform(X_train)
vectors_train_stop_Lemma = tfidf.fit_transform(vectors_train_stop_Lemma)

vectors_test_stop_Lemma = vectorizer.transform(X_test)
vectors_test_stop_Lemma = tfidf.transform(vectors_test_stop_Lemma)
vectors_train_stop_Lemma = normalizer_train.transform(vectors_train_stop_Lemma)
vectors_test_stop_Lemma = normalizer_train.transform(vectors_test_stop_Lemma)

print(vectorizer.get_feature_names())
print(vectors_train_stop_Lemma.shape)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make', 'u'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


['aa', 'aaa', 'aac', 'aarhus', 'abandon', 'abc', 'abdomen', 'abduct', 'abide', 'ability', 'abit', 'abl', 'ablative', 'able', 'abortion', 'abouyt', 'abroad', 'abruptly', 'absence', 'absent', 'absolute', 'absolutely', 'absorbance', 'absorbed', 'absorption', 'abstain', 'abstract', 'absurd', 'abt', 'abu', 'abundance', 'abundant', 'abuse', 'abusive', 'abusiveness', 'abyss', 'ac', 'aca', 'acad', 'academia', 'academic', 'academy', 'acaricide', 'accelerate', 'acceleration', 'accelerator', 'accelerometer', 'accent', 'accept', 'acceptable', 'accepted', 'accepts', 'access', 'accessibility', 'accessible', 'accessory', 'accident', 'accidental', 'accidentally', 'accidently', 'acclaim', 'acclimate', 'accommodate', 'accomodates', 'accompany', 'accomplish', 'accomplishment', 'accord', 'accordance', 'accordingly', 'account', 'accountable', 'accountant', 'accubattery', 'accumulate', 'accumulation', 'accuracy', 'accurate', 'accurately', 'accuse', 'accustom', 'ace', 'acer', 'achievable', 'achieve', 'achiev

In [21]:
class Bernoulli_NB():
  def __init__(self, LaplaceSmoothing = True):
    self.LaplaceSmoothing = LaplaceSmoothing
    self.Prob_Y = None      # P(Y)
    self.Prob_X_Y = None     # P(xj|Y)
    self.Prob_X_Y = None     # P(xj|Y)
    self.n_class = 0
    self.w0 = None
    self.w = None
    self.Ytarget = None


  def ProbY(self, Y):
    # calculate P(Y=1) and P(Y=0)
    ProbY = np.zeros((1,2))
    #print('shape',np.shape(Y))
    ProbY[0,1] = np.sum(Y)/np.shape(Y)
    ProbY[0,0] = 1 - ProbY[0,1]
    return ProbY

  def ProbX_Yi(self, X, Y, label):
    # calculte P(xj=1|Yi=1)
    rows,cols = X.shape       # feature shape
    numerator = np.zeros(cols)    # initialize numerator

    # xj=1 and Yi=1
    for n in range(rows):
      if Y[n] == label:
        numerator += X[n,:]
    # Yi=1
    denominator = np.count_nonzero(Y == label)
    # Laplace Smoothing
    if(self.LaplaceSmoothing):
      numerator += 1
      denominator += 2
    # P(xj=1|Yi=1)
    prob = numerator/denominator
    return prob    

  def fit(self, X, Y):
    print('----------------------start fitting---------------------')
    t1 = time.time()

    rows,cols = X.shape              # feature shape
    self.n_class = len(np.unique(Y))       # number of classes
    self.Prob_Y = np.zeros((self.n_class,2))   # initialize P(Y)
    self.Prob_X_Y = np.zeros((self.n_class,2,cols)) # initialize P(x|Y)
    c = np.zeros((self.n_class,cols))       # rows:class cols:xj
    d = np.zeros((self.n_class,cols))       # rows:class cols:xj
    self.w0 = np.zeros((1,self.n_class))     # [w0Y1,w0Y2,...]
    self.w = np.zeros((self.n_class,cols))    # [(w1,w2,...)Y1;
                            # (w1,w2,...)Y2]
    self.Ytarget = np.unique(Y)
    Y_index = 0
    for Yi in self.Ytarget:
      Y_onevsall = np.where(Y == Yi, 1, 0)    # only have 2 classes: Yi(1) & notYi(0)
      self.Prob_Y[Y_index,:] = self.ProbY(Y_onevsall)   # [P(notYi), P(Yi)]
      self.Prob_X_Y[Y_index,0,:] = self.ProbX_Yi(X,Y_onevsall,0)  # [P(x1|notYi), P(x2|notYi),...]
      self.Prob_X_Y[Y_index,1,:] = self.ProbX_Yi(X,Y_onevsall,1)  # [P(x1|Yi), P(x2|Yi),...]
      c[Y_index,:] = np.log10(self.Prob_X_Y[Y_index,1,:]/self.Prob_X_Y[Y_index,0,:])     # log(P(xj|Y=1)/P(xj|Y=0))
      d[Y_index,:] = np.log10((1-self.Prob_X_Y[Y_index,1,:])/(1-self.Prob_X_Y[Y_index,0,:])) # log((1-P(xj|Y=1))/(1-P(xj|Y=0)))
      self.w0[0,Y_index] = np.log10(self.Prob_Y[Y_index,1]/self.Prob_Y[Y_index,0]) + np.sum(d[Y_index,:])
      self.w[Y_index,:] = c[Y_index,:] - d[Y_index,:]
      Y_index += 1

    print('------fit done, total time:',time.time()-t1,' -----')
    # return self.Prob_Y,self.Prob_X_Y

  def predict(self, X):
    print('----------------------start predict---------------------')
    t1 = time.time()

    rows,cols = X.shape       # feature shape
    PreY = np.empty((rows,1),dtype='S') # initialize Y
    print('type',type(PreY))
    LogOddsRatio = np.zeros((1,self.n_class))  # initialize log odds ratio a(x) 
    Logistic = np.zeros((1,self.n_class))    # initialize logistic function
    Y_index = 0
    for obs in range(rows):
      for Yi in self.Ytarget:
        LogOddsRatio[0,Y_index] = self.w0[0,Y_index] + np.sum((self.w[Y_index,:]*X[obs,:])) 
        Logistic[0,Y_index] = 1/(1+np.exp(-LogOddsRatio[0,Y_index]))
        Y_index += 1
      Y_index = 0
      print(np.where(Logistic == np.amax(Logistic))[1])
      PreY[obs] = self.Ytarget[np.where(Logistic == np.amax(Logistic))[1]]
    print('------predict done, total time:',time.time()-t1,' -----')
    return PreY

  def Accu_eval(self,PreY,TrueY):
    #y_predict = self.predict(X_test)
    rows,cols = np.shape(PreY)
    TP = 0;FP = 0;TN = 0;FN = 0
    # count TP,TN,FP,FN in validation set
    for obs in range(rows):
      if  TrueY[obs]==1 and PreY[obs]==1:
        TP = TP+1
      elif TrueY[obs]==0 and PreY[obs]==0:
        TN = TN+1
      elif TrueY[obs]==0 and PreY[obs]==1:
        FP = FP+1
      elif TrueY[obs]==1 and PreY[obs]==0:
        FN = FN+1    
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F = 2*precision*recall/(precision+recall)
    specificity = TN/(FP+TN)
    FPR = FP/(FP+TN)
    print("---------------------accuracy:",accuracy,'-------------------')
    return accuracy


In [22]:
B = Bernoulli_NB()

parameters = {
...     'vect__ngram_range': [(1, 1), (1, 2)],
...     'tfidf__use_idf': (True, False),
... }

B.fit(vectors_train,Y_train)
#PreY = B.predict(vectors_test)
PreY = B.predict(vectors_train)
#B.Accu_eval(PreY,Ytest)

----------------------start fitting---------------------
------fit done, total time: 3.900989055633545  -----
----------------------start predict---------------------
type <class 'numpy.ndarray'>


ValueError: ignored